# NN with Bayesian Approximation

In [21]:
from collections import namedtuple
from numpy.random import uniform as U
import pandas as pd
import numpy as np
import io
import requests
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout

# Load data

In [22]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
s=requests.get(url).content
names = ['age', 
           'workclass', 
           'fnlwgt', 
           'education',
           'education_num',
           'marital_status',
           'occupation',
           'relationship',
           'race',
           'gender',
           'capital_gain',
           'capital_loss',
           'hours_per_week',
           'native_country',
          'income']
usecols = ['age', 
           'workclass', 
          'education',
           'marital_status',
           'occupation',
#            'relationship',
#            'race',
           'gender',
           'hours_per_week',
           'native_country',
           'income']
df_census = pd.read_csv(io.StringIO(s.decode('utf-8')), 
                        sep=',',
                        skipinitialspace=True,
                        names=names,
                        header=None,
                        usecols=usecols)

In [23]:
df_census.shape

(32561, 9)

In [24]:
df_census.head()

,age,workclass,education,marital_status,occupation,gender,hours_per_week,native_country,income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Male,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Female,40,Cuba,<=50K


# Prepocesssing

In [25]:
# Cleanup
df_census = df_census.replace('?', np.nan).dropna()
edu_map = {'Preschool': 'Elementary',
           '1st-4th': 'Elementary',
           '5th-6th': 'Elementary',
           '7th-8th': 'Elementary',
           '9th': 'Middle',
           '10th': 'Middle',
           '11th': 'Middle',
           '12th': 'Middle',
           'Some-college': 'Undergraduate',
           'Bachelors': 'Undergraduate',
           'Assoc-acdm': 'Undergraduate',
           'Assoc-voc': 'Undergraduate',
           'Prof-school': 'Graduate',
           'Masters': 'Graduate',
           'Doctorate': 'Graduate'}
for from_level, to_level in edu_map.items():
    df_census.education.replace(from_level, to_level, inplace=True)

In [26]:
df_census.head()

,age,workclass,education,marital_status,occupation,gender,hours_per_week,native_country,income
0,39,State-gov,Undergraduate,Never-married,Adm-clerical,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,Undergraduate,Married-civ-spouse,Exec-managerial,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Male,40,United-States,<=50K
3,53,Private,Middle,Married-civ-spouse,Handlers-cleaners,Male,40,United-States,<=50K
4,28,Private,Undergraduate,Married-civ-spouse,Prof-specialty,Female,40,Cuba,<=50K


In [27]:
df_census['native_country'].unique()

array(['United-States', 'Cuba', 'Jamaica', 'India', 'Mexico',
       'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany', 'Iran',
       'Philippines', 'Poland', 'Columbia', 'Cambodia', 'Thailand',
       'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'Italy', 'China', 'South', 'Japan', 'Yugoslavia', 'Peru',
       'Outlying-US(Guam-USVI-etc)', 'Scotland', 'Trinadad&Tobago',
       'Greece', 'Nicaragua', 'Vietnam', 'Hong', 'Ireland', 'Hungary',
       'Holand-Netherlands'], dtype=object)

In [28]:
# Convert raw data to processed data
context_cols = [c for c in usecols if c != 'education']
df_data = pd.concat([pd.get_dummies(df_census[context_cols]),df_census['education']], axis=1)

In [29]:
df_data.head()

,age,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Divorced,...,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K,education
0,39,40,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,Undergraduate
1,50,13,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,Undergraduate
2,38,40,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,HS-grad
3,53,40,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,Middle
4,28,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Undergraduate


In [30]:
# Let's start with determining the ad availability probabilities for each education 
# category and implement the sampling of ads. 
# As mentioned above, the ad server will have at most one ad for each target group. 
# We also ensure that there is at least one ad in the inventory.

def get_ad_inventory():
    ad_inv_prob = {'Elementary': 0.9, 
                   'Middle':  0.7, 
                   'HS-grad':  0.7, 
                   'Undergraduate':  0.9, 
                   'Graduate':  0.8}
    ad_inventory = []
    for level, prob in ad_inv_prob.items():
        if U() < prob:
            ad_inventory.append(level)
    # Make sure there are at least one ad among all 
    if not ad_inventory:
        ad_inventory = get_ad_inventory()
    return ad_inventory

In [31]:
# we define a function to generate a click probabilistically, 
# where the likelihood of a click increases to the degree that the user's education level and the ad's target match.

def get_ad_click_probs():
    base_prob = 0.8 # When an ad is shown to the user and if ad's target matches then probability of clicking on an ad
    delta = 0.4     # Probability decrease by 0.3 for each level of mismatch
    ed_levels = {'Elementary': 1, 
                 'Middle':  2, 
                 'HS-grad':  3, 
                 'Undergraduate':  4, 
                 'Graduate':  5}
    ad_click_probs = {l1: {l2: max(0, base_prob - delta * abs(ed_levels[l1]- ed_levels[l2])) for l2 in ed_levels}
                           for l1 in ed_levels}
    return ad_click_probs

# ad_click_probs:  {
# 'Elementary': {'Elementary': 0.8, 'Middle': 0.5, 'HS-grad': 0.20000000000000007, 'Undergraduate': 0, 'Graduate': 0}, 
# 'Middle': {'Elementary': 0.5, 'Middle': 0.8, 'HS-grad': 0.5, 'Undergraduate': 0.20000000000000007, 'Graduate': 0}, 
# 'HS-grad': {'Elementary': 0.20000000000000007, 'Middle': 0.5, 'HS-grad': 0.8, 'Undergraduate': 0.5, 'Graduate': 0.20000000000000007}, 
# 'Undergraduate': {'Elementary': 0, 'Middle': 0.20000000000000007, 'HS-grad': 0.5, 'Undergraduate': 0.8, 'Graduate': 0.5}, 
# 'Graduate': {'Elementary': 0, 'Middle': 0, 'HS-grad': 0.20000000000000007, 'Undergraduate': 0.5, 'Graduate': 0.8}}

In [32]:
# Generates a random click probility based on uniform random generator
def display_ad(ad_click_probs, user, ad):
    prob = ad_click_probs[ad][user['education']]
    click = 1 if U() < prob else 0 #U() Generate a random float number between 0 to 1
    return click

In [33]:
def calc_regret(user, ad_inventory, ad_click_probs, ad_selected):
    this_p = 0
    max_p = 0
    for ad in ad_inventory:
        p = ad_click_probs[ad][user['education']]
        if ad == ad_selected:
            this_p = p
        if p > max_p:
            max_p = p
    regret = max_p - this_p
    return regret

In [34]:
def get_model(n_input, dropout):
    inputs = keras.Input(shape=(n_input,))
    x = Dense(256, activation='relu')(inputs)
    if dropout > 0:
        x = Dropout(dropout)(x, training=True)
    x = Dense(256, activation='relu')(x)
    if dropout > 0:
        x = Dropout(dropout)(x, training=True)
    phat = Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, phat)
    model.compile(loss=keras.losses.BinaryCrossentropy(),
                  optimizer=keras.optimizers.Adam(),
                  metrics=[keras.metrics.binary_accuracy])
    return model

In [35]:
def update_model(model, X, y):
    X = np.array(X)
    X = X.reshape((X.shape[0], X.shape[2]))
    y = np.array(y).reshape(-1)
    model.fit(X, y, epochs=10)
    return model

In [36]:
# We then define a function that returns a one-hot representation for a specified ad based on the education level it targets. 
def ad_to_one_hot(ad):
    ed_levels = ['Elementary', 
                 'Middle', 
                 'HS-grad', 
                 'Undergraduate', 
                 'Graduate']
    ad_input = [0] * len(ed_levels)
    if ad in ed_levels:
        ad_input[ed_levels.index(ad)] = 1
    return ad_input

In [37]:
# We implement the Thompson sampling to select an ad given the context and the ad inventory at hand. 
def select_ad(model, context, ad_inventory):
    selected_ad = None
    selected_x = None
    max_action_val = 0
    for ad in ad_inventory:
        ad_x = ad_to_one_hot(ad)
        x = np.array(context + ad_x).reshape((1, -1))
        action_val_pred = model.predict(x)[0][0]
        if action_val_pred >= max_action_val:
            selected_ad = ad
            selected_x = x
            max_action_val = action_val_pred
    return selected_ad, selected_x

# The ad to be displayed is chosen based on the largest action value estimate that we obtain from the DNN. 
# We obtain this by trying all available ads in the inventory and remember, we have at most 
# one ad per target user group with the context of the user. 
# Note that the target user group is equivalent to action, which we feed to the DNN in the format of one-hot vector. 

In [38]:
def generate_user(df_data):
    user = df_data.sample(1)
    context = user.iloc[:, :-1].values.tolist()[0]
    return user.to_dict(orient='records')[0], context

# user is complete dataframe 
# user.to_dict(orient='records')[0]:  {'age': 44, 'hours_per_week': 40, 'workclass_Federal-gov': 0, 'workclass_Local-gov': 0, 'workclass_Private': 1, 'workclass_Self-emp-inc': 0,
# Context - Dataframe without class labels

In [39]:
df_data.head(2)

,age,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,marital_status_Divorced,...,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K,education
0,39,40,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,Undergraduate
1,50,13,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,Undergraduate


In [40]:
ad_click_probs = get_ad_click_probs()
df_cbandits = pd.DataFrame()
dropout_levels = [0.01]#, 0.05, 0.1, 0.2]
for d in dropout_levels:
    print("Trying with dropout:", d)
    np.random.seed(0)
    context_n = df_data.shape[1] - 1 # 86 columns except the last column which is label
    ad_input_n = df_data.education.nunique() # ad_input_n:  5
    model = get_model(context_n + ad_input_n, d) # def get_model(n_input, dropout):
    X = []
    y = []
    regret_vec = []
    total_regret = 0
    for i in range(1000):
        if i % 20 == 0:
            print("# of impressions:", i)
        user, context = generate_user(df_data)
        ad_inventory = get_ad_inventory()
        
#                                     ad_inventory :  ['Elementary', 'Middle', 'HS-grad', 'Undergraduate', 'Graduate']
#                                     ad_inventory :  ['Elementary', 'HS-grad', 'Undergraduate', 'Graduate']
#                                     ad_inventory :  ['Elementary', 'Middle', 'HS-grad', 'Undergraduate', 'Graduate']
#                                     ad_inventory :  ['Elementary', 'Middle', 'Undergraduate', 'Graduate']
#                                     ad_inventory :  ['Elementary', 'Middle', 'Undergraduate', 'Graduate']

        ad, x = select_ad(model, context, ad_inventory)
        click = display_ad(ad_click_probs, user, ad) # what is the prob that the add will be clicked even if it is correct
        regret = calc_regret(user, ad_inventory, ad_click_probs, ad)
        total_regret += regret
        regret_vec.append(total_regret)
        X.append(x)
        y.append(click)
        if (i + 1) % 500 == 0:
            print('Updating the model at', i+1)
            model = update_model(model, X, y)
            X = []
            y = []
            
    df_cbandits['dropout: '+str(d)] = regret_vec

Trying with dropout: 0.01
# of impressions: 0
# of impressions: 20
# of impressions: 40
# of impressions: 60
# of impressions: 80
# of impressions: 100
# of impressions: 120
# of impressions: 140
# of impressions: 160
# of impressions: 180
# of impressions: 200
# of impressions: 220
# of impressions: 240
# of impressions: 260
# of impressions: 280
# of impressions: 300
# of impressions: 320
# of impressions: 340
# of impressions: 360
# of impressions: 380
# of impressions: 400
# of impressions: 420
# of impressions: 440
# of impressions: 460
# of impressions: 480
Updating the model at 500
Epoch 1/10
16/16 [==============================] - 1s 4ms/step - loss: 0.8850 - binary_accuracy: 0.6180
Epoch 2/10
16/16 [==============================] - 0s 3ms/step - loss: 0.6624 - binary_accuracy: 0.6560
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 0.6240 - binary_accuracy: 0.6880
Epoch 4/10
16/16 [==============================] - 0s 4ms/step - loss: 0.6070 - binary_a

In [41]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

df_cbandits.iplot(dash = ['dash'],# 'solid', 'dashdot', 'dot'],
                  xTitle='Impressions', 
                  yTitle='Cumulative Regret')

model.summary()

In [45]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [46]:
load_model = keras.models.load_model("model")

In [ ]:
match=0
for i in range(100):
    user, context = generate_user(df_data)
    ad, x = select_ad(load_model, context, ad_inventory)
    click = display_ad(ad_click_probs, user, ad)
    #print(click,ad,user['education'])
    if click==1 and ad==user['education']:
        #print(click,ad,user['education'])
        match+=1
print(match)